# In silico Voting Experiments

Implementation of cultures and voting rules

Code uploaded to the github.

In [1]:
import pandas as pd
import sklearn
import math
import numpy as np
import random
random.seed(12)

# 2 Cultures

In [2]:
class Сulture:
    def __init__(self, n, K):
        self.n = n
        self.K = K
        
        # self.P - matrix with profiles
        self.P = np.zeros((self.n, self.K, self.K))
        
        # self.M - matrix with borda scores
        self.M = np.zeros((self.K, self.K))
        
        self.ctype = None
        self.cctype = None

# 2.1 Rousseauist cultures

In [3]:
class Rousseauist_culture(Сulture):
    # We assume that 1 > 2 > ... > K
    def setParams(self, alpha, beta):
        # alpha >= 0
        # beta >= 0
        self.alpha = alpha
        self.beta = beta
        self.ctype = "Rousseauist_culture"
    
    def createProfiles(self):
        tempP = np.zeros((self.n, self.K, self.K))
        for p in range(self.n):
            for k1 in range(self.K):
                for k2 in range(k1+1, self.K):
                    if( random.random() < self.getP(k1,k2)):
                        tempP[p, k1, k2] = 1
                    else:
                        tempP[p, k2,k1] = 1 
        self.P = tempP
        return tempP
        
    def getP(self, k1, k2):
        # Truchon and Drissi-Bakhkhat
        # k > k'
        if(k2 <= k1): 
            raise NameError("k >= k'")
            
        s = math.exp(self.alpha + self.beta*(k2-k1-1))
        return s/(1+s)

In [4]:
r = Rousseauist_culture(3,4)
r.setParams(1,1)
P = r.createProfiles()
print(f'P: \n {P}')

P: 
 [[[0. 1. 1.]
  [0. 0. 1.]
  [0. 0. 0.]]

 [[0. 1. 1.]
  [0. 0. 1.]
  [0. 0. 0.]]

 [[0. 1. 1.]
  [0. 0. 1.]
  [0. 0. 0.]]]


# 2.2 Impartial culture

In [5]:
class Impartial_culture(Сulture):
    def setParams(self):
        self.ctype = "Impartial_culture"
    
    def createProfiles(self):
        
        tempP = np.zeros((self.n, self.K, self.K))
        for p in range(self.n):
            profile = list(range(self.K))
            random.shuffle(profile)
            for k1 in range(self.K):
                pos1 = profile[k1]
                for k2 in range(k1+1, self.K):
                    pos2 = profile[k2]
                    tempP[p, pos1, pos2] = 1
        self.P = tempP
        return tempP   

        

In [6]:
i = Impartial_culture(5,6)
i.setParams()
P= i.createProfiles()
print(f'P: \n {P}')

P: 
 [[[0. 1. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1. 0.]
  [1. 1. 0. 0. 1. 0.]
  [1. 1. 1. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 0.]]

 [[0. 1. 1. 1. 0. 1.]
  [0. 0. 1. 1. 0. 0.]
  [0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 0. 1.]
  [0. 1. 1. 1. 0. 0.]]

 [[0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 0. 1. 0.]
  [1. 1. 0. 1. 1. 1.]
  [1. 1. 0. 0. 1. 1.]
  [1. 0. 0. 0. 0. 0.]
  [1. 1. 0. 0. 1. 0.]]

 [[0. 0. 0. 1. 0. 1.]
  [1. 0. 1. 1. 1. 1.]
  [1. 0. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 1.]
  [1. 0. 0. 1. 0. 1.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 1. 0. 0. 1. 1.]
  [0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 1. 1. 1.]
  [1. 1. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0.]]]


# 2.3 Distributive cultures

Everything is fine except for simple Gini

In [7]:
class Distributive_culture(Сulture):
    def setParams(self):
        self.ctype = "Distributive_culture"
    
#     def sharesIntoLinear(self, shares):
#         # [0.61, 0.85, 0.42, 0.13]: [1, 0, 2, 3]: 1 > 0 > 2 > 3 
#         linear_pr = [i for (v, i) in sorted([(v, i) for (i, v) in enumerate(shares)], reverse=True)]
#         return linear_pr
    
    def GiniIndex(self, c_pr: np.array):
        # Done
        growing = sorted(c_pr)
        accumulated = np.cumsum(growing)
        S = 0
        for i in range(len(accumulated)):
            S += ((i+1)/len(accumulated) - accumulated[i])
        S *= 2/len(accumulated)
        return S
    
    def simpleGiniIndex(self, c_pr: np.array):
        # Problem
        n = len(c_pr)
        growing = sorted(c_pr)
        accumulated = [0] + list(np.cumsum(growing))
        S = 0
        for i in range(1,n+1):
            S += ( (i-0.5) / n - (accumulated[i]+accumulated[i-1]) / 2)
        S *= 2/n
        return S
        
    def standartDeviation(self, c_pr: np.array):
        # Done
        return np.std(c_pr)

# 2.3.1 Consensual redistributive culture

In [8]:
# Consensual redistributive culture
class Consensual_redistributive_culture(Distributive_culture):
    def setParams(self):
        self.ctype = "Distributive_culture"
        self.cctype = "Consensual_redistributive_culture"
        
        
    def createSharesProfiles(self):
        Y = []
        for i in range(self.n):
            Y.append(np.random.random((self.K)))
        X = []
        for i in range(self.n):
            X.append([])
            for j in range(self.K):
                X[-1].append(Y[i][j]/sum([Y[t][j] for t in range(self.n)]))
        return X
        
    def createProfiles(self):
        self.SharesProfiles = self.createSharesProfiles()
#         print(f'X : {np.matrix(X)}')
        tempP = np.zeros((self.n, self.K, self.K))
        for p in range(self.n):
            for k1 in range(self.K):
                for k2 in range(k1+1, self.K):
                    if(self.SharesProfiles[p][k1] >= self.SharesProfiles[p][k2]):
                        tempP[p, k1, k2] = 1
                    else:
                        tempP[p, k2, k1] = 1
        self.P = tempP
        return tempP   

In [9]:
crc = Consensual_redistributive_culture(n = 4, K = 6)
crc.setParams()
P= crc.createProfiles()
print(f'P: \n {P}')

P: 
 [[[0. 1. 1. 1. 1. 1.]
  [0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 1. 1. 1. 0. 1.]
  [0. 1. 1. 1. 0. 0.]]

 [[0. 0. 0. 1. 1. 0.]
  [1. 0. 0. 1. 1. 1.]
  [1. 1. 0. 1. 1. 1.]
  [0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 0. 0. 1. 1. 0.]]

 [[0. 0. 0. 0. 0. 1.]
  [1. 0. 1. 0. 0. 1.]
  [1. 0. 0. 0. 0. 1.]
  [1. 1. 1. 0. 1. 1.]
  [1. 1. 1. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 1. 1. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0.]
  [1. 1. 1. 0. 1. 1.]
  [0. 1. 1. 0. 0. 0.]
  [1. 1. 1. 0. 1. 0.]]]


In [10]:
def SimulateTable1(times = 100):
#     vals = [3,5,11,49,99,999]
    vals = [3,5,11,49,99]
    gini_avgs = ['Gini index']
    simple_gini_avgs = ['Simple Gini index']
    standart_deviations = ['Standart deviation']

    for n in vals:
        gini_avg = 0
        simple_gini_avg = 0
        standart_deviation = 0
        for t in range(times):
            crc = Consensual_redistributive_culture(n,1)
            crc.setParams()
            P = crc.createProfiles()
            shares = []
            for p in crc.SharesProfiles:
                shares.append(p[0])
            gini_avg += crc.GiniIndex(shares)
            simple_gini_avg += crc.simpleGiniIndex(shares)
            standart_deviation += crc.standartDeviation(shares)
        gini_avg /= times
        gini_avgs.append(gini_avg)
        simple_gini_avg /= times
        simple_gini_avgs.append(simple_gini_avg)
        standart_deviation /= times
        standart_deviations.append(standart_deviation)
    df = pd.DataFrame([standart_deviations, simple_gini_avgs, gini_avgs],columns=['n'] + vals)
    return df

df = SimulateTable1()
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)
print(f"\t\t\t \033[1m  Table 1  \033[0m")
print(df)


			   Table 1  
                    n       3       5      11      49      99
0  Standart deviation  0.1643  0.1048  0.0506  0.0114  0.0058
1   Simple Gini index  0.2562  0.2781  0.3097  0.3203  0.3303
2          Gini index  0.2562  0.2781  0.3097  0.3203  0.3303


# 2.3.2 Inegalitarian distributive cultures

In [11]:
# Inegalitarian distributive cultures
class Inegalitarian_distributive_culture(Distributive_culture):
    def setParams(self, e_min, e_max):
        self.ctype = "Distributive_culture"
        self.cctype = "Inegalitarian_distributive_culture"
        self.e_min = e_min
        self.e_max = e_max
        
        e = []
        for k in range(self.K):
            e.append(random.uniform(self.e_min, self.e_max))
    
    # ====================
    def createSharesProfiles(self):
        X = [[] for i in range(self.n)]
        e = []
        # define concentration parameter for each k:
        for k in range(self.K):
            e.append(random.uniform(self.e_min, self.e_max))
        for k in range(self.K):
            shares = self.defineShares(e[k])
            random.shuffle(shares)
            for i in range(self.n):
                X[i].append(shares[i])
        return X
        
    def createProfiles(self):
        self.SharesProfiles = self.createSharesProfiles()
        tempP = np.zeros((self.n, self.K, self.K))
        for p in range(self.n):
            for k1 in range(self.K):
                for k2 in range(k1+1, self.K):
                    if(self.SharesProfiles[p][k1] >= self.SharesProfiles[p][k2]):
                        tempP[p, k1, k2] = 1
                    else:
                        tempP[p, k2, k1] = 1
        self.P = tempP
        return tempP   

        
    def defineShares(self, e):
        v = [0] + [ ( (i+1) / self.n ) ** e  for i in range(self.n)]
        u = [v[i+1]-v[i] for i in range(self.n)]
        return u

In [12]:
crc = Inegalitarian_distributive_culture(4,6)
e_min = 2
e_max = 4

crc.setParams(e_min, e_max)
P= crc.createProfiles()
np.set_printoptions(precision=3)
print(f'SharesProfiles: {np.matrix(crc.SharesProfiles)}')
print(f'P: \n {P}')



SharesProfiles: [[0.04  0.009 0.179 0.039 0.007 0.553]
 [0.16  0.086 0.454 0.49  0.648 0.123]
 [0.313 0.282 0.313 0.313 0.074 0.021]
 [0.488 0.623 0.054 0.158 0.271 0.303]]
P: 
 [[[0. 1. 0. 1. 1. 0.]
  [0. 0. 0. 0. 1. 0.]
  [1. 1. 0. 1. 1. 0.]
  [0. 1. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0.]
  [1. 1. 1. 1. 1. 0.]]

 [[0. 1. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0.]
  [1. 1. 0. 0. 0. 1.]
  [1. 1. 1. 0. 0. 1.]
  [1. 1. 1. 1. 0. 1.]
  [0. 1. 0. 0. 0. 0.]]

 [[0. 1. 0. 1. 1. 1.]
  [0. 0. 0. 0. 1. 1.]
  [1. 1. 0. 1. 1. 1.]
  [0. 1. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 1.]
  [0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 1. 1. 1.]
  [1. 0. 1. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0.]
  [0. 0. 1. 1. 0. 0.]
  [0. 0. 1. 1. 1. 0.]]]


# 2.4 Spatial cultures


In [13]:
class Spatial_culture(Сulture):
    def setParams(self, d, sizes: list):
        self.d = d # dimentions
        self.sizes = sizes
        self.ctype = "Spatial_culture"
        
    def utility(self, w, x):
        return - np.linalg.norm(np.asarray(x) - np.asarray(w), ord=2)
        
    def createSpace(self):
        bliss_points = []
        for i in range(self.n):
            bp = [0] * len(self.sizes)
            for t in range(len(self.sizes)):
                bp[t] = random.uniform(0, self.sizes[t])
            bliss_points.append(bp)
        candidate_points = []
        for k in range(self.K):
            cp = [0] * len(self.sizes)
            for t in range(len(self.sizes)):
                cp[t] = random.uniform(0, self.sizes[t])
            candidate_points.append(cp)
        return bliss_points, candidate_points
            
        
        
    def createProfiles(self):
        self.bliss_points, self.candidate_points = self.createSpace()
        
        tempP = np.zeros((self.n, self.K, self.K))
        for p in range(self.n):
            bp = self.bliss_points[p]
            utilities = []
            for cp in self.candidate_points:
                utilities.append(self.utility(bp, cp))
            for k1 in range(self.K):
                ut1 = utilities[k1]
                for k2 in range(k1+1, self.K):
                    ut2 = utilities[k2]
                    if(ut1>ut2):
                        tempP[p, k1, k2] = 1
                    else:
                        tempP[p, k2, k1] = 1
        self.P = tempP
        return tempP   
    
    

# 3 Voting rules and behavior

In [14]:
class Voting:
    def __init__(self, culture):
        self.n = culture.n
        self.K = culture.K
        self.vtype = None
        self.culture = culture
        self.setParams()
        
    def checkTies(self, scores: list):
        if len(scores) == len(set(scores)):
            return False
        else:
            return True
        
    def addNoise(self, scores: list):
        noise = np.random.rand(len(scores))/1000
        scores += noise
        return scores

# 3.1 Sincere voting

In [15]:
class SincereVoting(Voting):
    def setParams(self):
        self.vtype = 'SincereVoting'
        self.P = self.culture.P
        
    def PluralityScores(self):
        PluralityScore = np.zeros(self.K)
        for p in range(self.n):
            tempPS = np.zeros(self.K)
            scores = np.sum(self.P[p], axis = 1)
            tempBests = np.argwhere(scores == np.amax(scores))
            for best in tempBests:
                tempPS[best] += 1/len(tempBests)
            PluralityScore += tempPS
        return PluralityScore
    
    def PluralityWinner(self):
        ps = self.PluralityScores()
        winner = np.argmax(ps)
        return winner
        
    def createBordaMatrix(self):
        bordaMatrix = np.zeros((self.K, self.K))
        for p in range(self.n):
            bordaMatrix += self.P[p]
        for k in range(self.K):
            bordaMatrix[k,k] = self.n/2
        return bordaMatrix
    
    def BordaScores(self):
        self.bordaMatrix = self.createBordaMatrix()
        bs = []
        for k1 in range(self.K):
            bs.append(0)
            for k2 in range(self.K):
                bs[k1] += self.bordaMatrix[k1][k2]
        return bs
    
    def BordaWinner(self):
        bs = self.BordaScores()
        winner = np.argmax(bs)
        return winner

    def CopelandScores(self):
        self.bordaMatrix = self.createBordaMatrix()
        self.tournamentMatrix = [[int(el > self.n/2) for el in line] for line in self.bordaMatrix]
        cs = []
        for k1 in range(self.K):
            cs.append(0)
            for k2 in range(self.K):
                cs[k1] += self.tournamentMatrix[k1][k2]
        return cs
    
    def CopelandWinner(self):
        cs = self.CopelandScores()
        winner = np.argmax(cs)
        return winner
    
    def checkCondorsetWinner(self):
        cs = self.CopelandScores()
        if(self.K-1 in cs):
            return True, np.argmax(cs)
        return False, -1

In [16]:
# r = Rousseauist_culture(4,10)
r = Rousseauist_culture(3,5)
r.setParams(0,0.2)
P = r.createProfiles()
print(f'P: \n {P}')

# If there are fractions in answer, then preferences aren't strict.
SV = SincereVoting(r)
print(f'Plurality Score: {SV.PluralityScores()}')
print(f'PluralityWinner: {SV.checkCondorsetWinner()}')
print(f'Borda Score: {SV.BordaScores()}')
print(f'BordaWinner: {SV.BordaWinner()}')
print(f'Copeland Score: {SV.CopelandScores()}')
print(f'CondorsetWinner: {SV.checkCondorsetWinner()}')

P: 
 [[[0. 1. 1. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 1. 0. 1. 1.]
  [0. 0. 0. 0. 1.]
  [1. 1. 0. 0. 0.]]

 [[0. 1. 1. 0. 1.]
  [0. 0. 0. 1. 0.]
  [0. 1. 0. 1. 1.]
  [1. 0. 0. 0. 1.]
  [0. 1. 0. 0. 0.]]

 [[0. 0. 1. 1. 1.]
  [1. 0. 1. 1. 0.]
  [0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 1.]
  [0. 1. 1. 0. 0.]]]
Plurality Score: [1.5 0.5 1.  0.  0. ]
PluralityWinner: 0
Borda Score: [10.5, 6.5, 8.5, 5.5, 6.5]
BordaWinner: 0
Copeland Score: [4, 1, 3, 1, 1]
CondorsetWinner: (True, 0)


# 3.2 Responsive voting

In [17]:
class ResponsiveVoting(Voting):
    def setParams(self):
        self.vtype = 'ResponsiveVoting'
        self.P = self.culture.P
    
    
    def PluralityScores(self):
        # We always assume that announced scores is 0 > 1 > 2 > ... > K-1
        PluralityScore = np.zeros(self.K)
        for p in range(self.n):
            tempPS = np.zeros(self.K)
            scores = np.sum(self.P[p], axis = 1)
            tempBests = np.argwhere(scores == np.amax(scores))
            for best in tempBests:
                tempPS[best] += 1/len(tempBests)
            PluralityScore += tempPS
        return PluralityScore
    def PluralityWinner(self):
        ps = self.PluralityScores()
        winner = np.argmax(ps)
        return winner
        
    def createBordaMatrix(self):
        bordaMatrix = np.zeros((self.K, self.K))
        for p in range(self.n):
            bordaMatrix += self.P[p]
        for k in range(self.K):
            bordaMatrix[k,k] = self.n/2
        return bordaMatrix
    
    def BordaScores(self, initial_scores = []):
        #Starting with Borda ranking
        if(len(initial_scores)==0):
            self.bordaMatrix = self.createBordaMatrix()
            bs = []
            for k1 in range(self.K):
                bs.append(0)
                for k2 in range(self.K):
                    bs[k1] += self.bordaMatrix[k1][k2]

            if(self.checkTies(bs)):
                bs = self.addNoise(bs)
        else:
            bs = initial_scores
            
        bs_dict = {}
        bs_pairs = []
        for i, el in enumerate(bs):
            bs_dict[i] = el
            bs_pairs.append((i, el))
            
        bs_pairs.sort(key = lambda x: x[1], reverse=True)
        bs_keys_sorted = [el[0] for el in bs_pairs]
        leader = bs_keys_sorted[0]
        challenger = bs_keys_sorted[1]
        
        borda_ballots = []
        NEW_BORDA_MATRIX = np.zeros((self.K, self.K))
        for p in range(self.n):
            profileMatrix = np.zeros((self.K, self.K))
            ballot = [0]*self.K
            if(self.P[p][leader][challenger]==1):
                ballot[0] = leader
                ballot[self.K-1] = challenger 
            else:
                ballot[0] = challenger
                ballot[self.K-1] = leader

            st_pointer = 1
            end_pointer = self.K-2
            
            for k2, chal in enumerate(bs_keys_sorted):
                if(k2>1):
                    if(self.P[p][leader][chal]==1):
                        ballot[end_pointer] = chal
                        end_pointer -=1
                    else:
                        ballot[st_pointer] = chal
                        st_pointer += 1
#
            # ballot  = [1,5,2,3,0] ~ 1>5>2>3>0
            result_ballot = [0]*self.K
            for k, el in enumerate(ballot):
                result_ballot[el] = self.K-1 - k
            borda_ballots.append(result_ballot)
        borda_voting = [0]*self.K
        for bal in borda_ballots:
            for k in range(len(bal)):
                borda_voting[k]+=bal[k]
        return borda_voting
            

    def BordaWinner(self):
        bs = self.BordaScores()
        winner = np.argmax(bs)
        return winner

    def CopelandScores(self):
        self.bordaMatrix = self.createBordaMatrix()
        self.tournamentMatrix = [[int(el > self.n/2) for el in line] for line in self.bordaMatrix]
        cs = []
        for k1 in range(self.K):
            cs.append(0)
            for k2 in range(self.K):
                cs[k1] += self.tournamentMatrix[k1][k2]
        return cs
    
    def CopelandWinner(self):
        cs = self.CopelandScores()
        winner = np.argmax(cs)
        return winner
    
    def checkCondorsetWinner(self):
        cs = self.CopelandScores()
        if(self.K-1 in cs):
            return True, np.argmax(cs)
        return False, -1
    
    def ApprovalVoting(self, initial_scores = None):
        #Starting with Copeland ranking
        cs = initial_scores
        if(initial_scores == None):
            cs = self.CopelandScores()
        if(self.checkTies(cs)):
            cs = self.addNoise(cs)
#         print(f'cs: {cs}')
        leader = np.argmax(cs) 
        av = []
        for p in range(self.n):
            ballot = [0]*self.K
            if(sum(self.P[p][leader]) == self.K - 1):
                ballot[leader] = 1
            else:
                for k in range(self.K):
                    if(k != leader):
                        if(self.P[p][k][leader]==1):
                            ballot[k] = 1
                            
            av.append(ballot)
#         print(f'av: {av}')
        res = [0]*self.K
        for p in range(self.n):
            for k in range(self.K):
                res[k] += av[p][k]
        return res           
        
        

In [18]:
# r = Rousseauist_culture(4,10)
r = Rousseauist_culture(3,7)
r.setParams(0.4,0.5)
P = r.createProfiles()
print(f'P: \n {P}')

# If there are fractions in answer, then preferences aren't strict.
RV = ResponsiveVoting(r)
# print(f'Plurality Score: {SV.PluralityScores()}')
print(f'\nBorda scores: {RV.BordaScores()}\n')
print(f'AV Score: {RV.ApprovalVoting()}')
# print(f'Copeland Score: {SV.CopelandScores()}')
# print(f'CondorsetWinner: {SV.checkCondorsetWinner()}')

P: 
 [[[0. 0. 1. 1. 1. 1. 1.]
  [1. 0. 1. 0. 1. 1. 1.]
  [0. 0. 0. 1. 1. 0. 0.]
  [0. 1. 0. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 1. 0. 0.]
  [0. 0. 1. 0. 1. 1. 0.]]

 [[0. 1. 1. 1. 1. 1. 1.]
  [0. 0. 0. 1. 1. 0. 1.]
  [0. 1. 0. 0. 1. 1. 1.]
  [0. 0. 1. 0. 1. 1. 1.]
  [0. 0. 0. 0. 0. 0. 1.]
  [0. 1. 0. 0. 1. 0. 1.]
  [0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 1. 1. 1. 0. 1.]
  [1. 0. 0. 0. 0. 1. 1.]
  [0. 1. 0. 1. 0. 1. 1.]
  [0. 1. 0. 0. 1. 1. 1.]
  [0. 1. 1. 0. 0. 1. 1.]
  [1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]]]

Borda scores: [18, 11, 4, 0, 9, 9, 12]

AV Score: [1, 2, 0, 0, 0, 1, 0]


# 4 Results

# 4.1 Results for Rousseauist cultures

### Tables 3 and 4 simulations:

In [19]:
def SimulateTable34(n, K, alpha, beta, simulation_runs):
    CondorChance = 0
    CondorRousseauChance = 0
    for i in range(simulation_runs):
        r = Rousseauist_culture(n,K)
        r.setParams(alpha, beta)
        P = r.createProfiles()
        v = SincereVoting(r)
        CW_exists, CW_num = v.checkCondorsetWinner()
        if(CW_exists):
            CondorChance += 1
            # We assume that truee order is 0 > 1 > 2 > ... > K -1
            # So Rousseau winner is 0
            CondorRousseauChance += int(CW_num == 0)
    # print(f'Probability for the Condorset winner  {CondorChance/simulation_runs}')
    return CondorChance/simulation_runs, CondorRousseauChance/simulation_runs

In [20]:
simulation_runs = 1000

n = 5
K_values = [5, 11]
alpha_values = [0, 0.4, 0.7, 1]
beta_values  = [0, 0.3, 0.5, 0.7, 1]

for K in K_values:
    print(f"\t\t\t \033[1m  n = {n} K = {K}  \033[0m")
    # print('')
    df = pd.DataFrame(columns=['beta = '+str(beta) for beta in beta_values])
    for alpha in alpha_values:
        line = []
        for beta in beta_values:
            CondorChance, CondorRousseauChance  = SimulateTable34(n, K, alpha, beta, simulation_runs)
            line.append(str(CondorChance) + ' (' +str(CondorRousseauChance) +')' )
        df.loc[f'alpha = {alpha}'] = line
    print(df)
    print('\n')


			   n = 5 K = 5  
                  beta = 0     beta = 0.3     beta = 0.5     beta = 0.7       beta = 1
alpha = 0    0.312 (0.065)  0.431 (0.223)  0.503 (0.266)  0.584 (0.343)  0.685 (0.449)
alpha = 0.4  0.397 (0.193)   0.65 (0.461)  0.713 (0.513)  0.807 (0.589)  0.862 (0.641)
alpha = 0.7  0.529 (0.379)  0.754 (0.615)  0.862 (0.701)  0.888 (0.741)   0.93 (0.764)
alpha = 1    0.711 (0.601)  0.864 (0.767)  0.935 (0.834)  0.941 (0.826)  0.968 (0.862)


			   n = 5 K = 11  
                  beta = 0     beta = 0.3     beta = 0.5     beta = 0.7       beta = 1
alpha = 0    0.009 (0.001)  0.273 (0.164)  0.483 (0.282)    0.635 (0.4)  0.653 (0.412)
alpha = 0.4   0.04 (0.021)   0.58 (0.419)  0.721 (0.517)  0.807 (0.605)  0.863 (0.662)
alpha = 0.7  0.123 (0.091)   0.754 (0.61)  0.869 (0.719)    0.9 (0.734)  0.934 (0.768)
alpha = 1    0.298 (0.261)  0.857 (0.749)   0.93 (0.831)  0.955 (0.854)  0.969 (0.873)




In [21]:
def SimulateTable56(n, K, alpha, beta, simulation_runs):
    CondorNum = 0
    CondorRousseauNum = 0
    
    CopelandWinner = 0
    AV1_win = []
    AV2_win = []
    AV3_win = []
    AV4_win = []
    AV5_win = []
    AV_5_iter = [0]*5
    
    BordaWinner = 0
    Borda1_win = []
    Borda2_win = []
    Borda3_win = []
    Borda4_win = []
    Borda5_win = []
    Borda_5_iter = [0]*5
    for i in range(simulation_runs):
        
        r = Rousseauist_culture(n,K)
        r.setParams(alpha, beta)
        P = r.createProfiles()
        SV = SincereVoting(r)
        RV = ResponsiveVoting(r)
        
        CW_exists, CW_num = SV.checkCondorsetWinner()
        
        if(CW_exists):
            CondorNum += 1
            # We assume that truee order is 0 > 1 > 2 > ... > K -1
            # So Rousseau winner is always 0
            CondorRousseauNum += int(CW_num == 0)
    
        CopelandWinner += 1 + SV.CopelandWinner()
        BordaWinner += 1 + SV.BordaWinner()
    
        Borda1_scores = RV.BordaScores()
        Borda2_scores = RV.BordaScores(Borda1_scores)
        Borda3_scores = RV.BordaScores(Borda2_scores)
        Borda4_scores = RV.BordaScores(Borda3_scores)
        Borda5_scores = RV.BordaScores(Borda4_scores)
        
        Borda1_win.append(1 + np.argmax(Borda1_scores))
        Borda2_win.append(1 + np.argmax(Borda2_scores))
        Borda3_win.append(1 + np.argmax(Borda3_scores))
        Borda4_win.append(1 + np.argmax(Borda4_scores))
        Borda5_win.append(1 + np.argmax(Borda5_scores))
        
        AV1_scores = RV.ApprovalVoting()
        AV2_scores = RV.ApprovalVoting(AV1_scores)
        AV3_scores = RV.ApprovalVoting(AV2_scores)
        AV4_scores = RV.ApprovalVoting(AV3_scores)
        AV5_scores = RV.ApprovalVoting(AV4_scores)
        
        AV1_win.append(1 + np.argmax(AV1_scores))
        AV2_win.append(1 + np.argmax(AV2_scores))
        AV3_win.append(1 + np.argmax(AV3_scores))
        AV4_win.append(1 + np.argmax(AV4_scores))
        AV5_win.append(1 + np.argmax(AV5_scores))
            
    
    CondorCh = CondorNum/simulation_runs
    CondorRousseauCh = CondorRousseauNum/simulation_runs
    
    
    CopelandAvg = CopelandWinner /simulation_runs
    BordaWinner = BordaWinner /simulation_runs
    
    Borda_5_iter[0] = np.mean(Borda1_win)
    Borda_5_iter[1] = np.mean(Borda2_win)
    Borda_5_iter[2] = np.mean(Borda3_win)
    Borda_5_iter[3] = np.mean(Borda4_win)
    Borda_5_iter[4] = np.mean(Borda5_win)
    
    AV_5_iter[0] =  np.mean(AV1_win)
    AV_5_iter[1] =  np.mean(AV2_win)
    AV_5_iter[2] =  np.mean(AV3_win)
    AV_5_iter[3] =  np.mean(AV4_win)
    AV_5_iter[4] =  np.mean(AV5_win)
    
    return CondorCh, CondorRousseauCh, CopelandAvg, AV_5_iter,BordaWinner, Borda_5_iter

In [22]:
# Table 5
simulation_runs = 1000

n_values = [5,11]
K = 5
alpha = 0.4
beta  = 0.5
print(f"\t\t\t \033[1m  alpha =.4, beta=.5, K=5  \033[0m")
df = pd.DataFrame(columns=['n = 5', 'n = 11'])
# df = pd.DataFrame(columns=['\033[1m alpha =.4, beta=.5, K=5 \033[0m', 'n = 5', 'n = 11'])
cols = {}
for n in n_values:
    cols[f'n = {n}']={}
    CondorCh, CondorRousseauCh, CopelandR, AV, BordaWinner, bordaTop5Avg  = SimulateTable56(n, K, alpha, beta, simulation_runs)
    cols[f'n = {n}']['Pr. of Condorcet :'] = CondorCh
    cols[f'n = {n}']['Pr. of Rousseau-Condorcet :'] = CondorRousseauCh
    cols[f'n = {n}']['Rule'] = 'Average rank'
    cols[f'n = {n}']['Copeland'] = CopelandR
    for i in range(len(AV)):
        cols[f'n = {n}'][f'AV{i+1}'] = AV[i]
    cols[f'n = {n}']['Borda'] = BordaWinner
    for i in range(len(bordaTop5Avg)):
        cols[f'n = {n}'][f'Borda{i+1}'] = bordaTop5Avg[i]
    for key in cols[f'n = {n}'].keys():
        df.at[key,f'n = {n}'] = cols[f'n = {n}'][key]
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)
print(df)


			   alpha =.4, beta=.5, K=5  
                                    n = 5        n = 11
Pr. of Condorcet :                  0.748         0.872
Pr. of Rousseau-Condorcet :         0.565         0.685
Rule                         Average rank  Average rank
Copeland                            1.238         1.202
AV1                                 1.691         1.783
AV2                                 1.707         1.449
AV3                                 1.473         1.492
AV4                                 1.603         1.574
AV5                                 1.673         1.606
Borda                               1.238         1.141
Borda1                              2.696         2.911
Borda2                               1.55         1.327
Borda3                              2.201         2.184
Borda4                               1.51         1.371
Borda5                              1.918         1.663



# 4.2 Results for impartial cultures

In [23]:
def SimulateTable7(n, K, simulation_runs):
    CondorChance = 0
#     CondorRousseauChance = 0
    for i in range(simulation_runs):
        r = Impartial_culture(n,K)
        r.setParams()
        P = r.createProfiles()
        v = SincereVoting(r)
        CW_exists, CW_num = v.checkCondorsetWinner()
        if(CW_exists):
            CondorChance += 1
            # We assume that truee order is 0 > 1 > 2 > ... > K -1
            # So Rousseau winner is 0
#             CondorRousseauChance += int(CW_num == 0)
    # print(f'Probability for the Condorset winner  {CondorChance/simulation_runs}')
    return CondorChance/simulation_runs #, CondorRousseauChance/simulation_runs

In [24]:
simulation_runs = 1000

n_values = [3,5,11,99]
K_values = [3,5, 15, 50]

print(f"\t\t\t \033[1m  Table 7  \033[0m")
df = pd.DataFrame(columns=['n = '+str(n) for n in n_values])
for K in K_values:
    line = []
    for n in n_values:    
        CondorChance  = SimulateTable7(n, K, simulation_runs)
        line.append(str(CondorChance))
    df.loc[f'K = {K}'] = line
print(df)
print('\n')


			   Table 7  
        n = 3  n = 5 n = 11 n = 99
K = 3   0.949  0.928  0.926   0.92
K = 5    0.82  0.808  0.757  0.732
K = 15  0.559  0.515  0.429  0.372
K = 50  0.355  0.269  0.168   0.17




In [25]:
def SimulateTable8(n, K, simulation_runs):
    CondorNum = 0
    CopelandEqBorda = 0
    PluralityEqBorda = 0 
    
    AV1_win = []
    AV2_win = []
    AV3_win = []
    AV4_win = []
    AV5_win = []
    AV_5_iter = [0]*5

    Borda1_win = []
    Borda2_win = []
    Borda3_win = []
    Borda4_win = []
    Borda5_win = []
    Borda_5_iter = [0]*5
    for i in range(simulation_runs):
            
        r = Impartial_culture(n,K)
        r.setParams()
        P = r.createProfiles()
        SV = SincereVoting(r)
        RV = ResponsiveVoting(r)
        
        CW_exists, CW_num = SV.checkCondorsetWinner()
        
        if(CW_exists):
            CondorNum += 1
    
        CopelandWinner = 1 + SV.CopelandWinner()
        BordaWinner = 1 + SV.BordaWinner()
        PluralityWinner = 1 + SV.PluralityWinner() 
        CopelandEqBorda += int(CopelandWinner == BordaWinner)
        PluralityEqBorda += int(PluralityWinner == BordaWinner)
        
        
        Borda1_scores = RV.BordaScores()
        Borda2_scores = RV.BordaScores(Borda1_scores)
        Borda3_scores = RV.BordaScores(Borda2_scores)
        Borda4_scores = RV.BordaScores(Borda3_scores)
        Borda5_scores = RV.BordaScores(Borda4_scores)
        
        Borda1_win.append(1 + np.argmax(Borda1_scores) == BordaWinner)
        Borda2_win.append(1 + np.argmax(Borda2_scores) == BordaWinner)
        Borda3_win.append(1 + np.argmax(Borda3_scores) == BordaWinner)
        Borda4_win.append(1 + np.argmax(Borda4_scores) == BordaWinner)
        Borda5_win.append(1 + np.argmax(Borda5_scores) == BordaWinner)
        
        AV1_scores = RV.ApprovalVoting()
        AV2_scores = RV.ApprovalVoting(AV1_scores)
        AV3_scores = RV.ApprovalVoting(AV2_scores)
        AV4_scores = RV.ApprovalVoting(AV3_scores)
        AV5_scores = RV.ApprovalVoting(AV4_scores)
        
        AV1_win.append(1 + np.argmax(AV1_scores) == BordaWinner )
        AV2_win.append(1 + np.argmax(AV2_scores) == BordaWinner)
        AV3_win.append(1 + np.argmax(AV3_scores) == BordaWinner)
        AV4_win.append(1 + np.argmax(AV4_scores) == BordaWinner)
        AV5_win.append(1 + np.argmax(AV5_scores) == BordaWinner)
            
    
    CondorCh = CondorNum/simulation_runs
    CopelandCh = CopelandEqBorda/simulation_runs
    Plurality = PluralityEqBorda/simulation_runs
    
    
    Borda_5_iter[0] = np.mean(Borda1_win)
    Borda_5_iter[1] = np.mean(Borda2_win)
    Borda_5_iter[2] = np.mean(Borda3_win)
    Borda_5_iter[3] = np.mean(Borda4_win)
    Borda_5_iter[4] = np.mean(Borda5_win)
    
    AV_5_iter[0] =  np.mean(AV1_win)
    AV_5_iter[1] =  np.mean(AV2_win)
    AV_5_iter[2] =  np.mean(AV3_win)
    AV_5_iter[3] =  np.mean(AV4_win)
    AV_5_iter[4] =  np.mean(AV5_win)
    
    return CondorCh, Plurality, CopelandCh, AV_5_iter, Borda_5_iter

In [26]:
# Table 8
print(f"\t\t\t \033[1m  Table 8  \033[0m")

simulation_runs = 1000

n = 11
K_values = [3,5,15]
print(f"\t\t\t \033[1m  n = 11  \033[0m")
df = pd.DataFrame(columns=['K = 3', 'K = 5', 'K = 15'])
cols = {}
for k in K_values:
    cols[f'K = {k}']={}
    CondorCh, Plurality, CopelandCh, AV, Borda  = SimulateTable8(n, k, simulation_runs)
    cols[f'K = {k}']['Pr. of Condorcet :'] = CondorCh
    cols[f'K = {k}']['Rule'] = 'Borda'
    cols[f'K = {k}']['Plurality'] = Plurality
    cols[f'K = {k}']['Copeland:'] = CopelandCh
    for i in range(len(AV)):
        cols[f'K = {k}'][f'AV{i+1}'] = AV[i]
    cols[f'K = {k}']['Borda'] = 1
    for i in range(len(Borda)):
        cols[f'K = {k}'][f'Borda{i+1}'] = Borda[i]
    for key in cols[f'K = {k}'].keys():
        df.at[key,f'K = {k}'] = cols[f'K = {k}'][key]
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)
print(df)


			   Table 8  
			   n = 11  
                    K = 3  K = 5 K = 15
Pr. of Condorcet :  0.919  0.777  0.456
Rule                Borda  Borda  Borda
Plurality           0.796  0.573  0.278
Copeland:           0.898  0.779  0.698
AV1                 0.605  0.262  0.088
AV2                 0.782  0.508  0.194
AV3                 0.664   0.47  0.386
AV4                 0.744  0.482  0.273
AV5                 0.677  0.394  0.224
Borda                   1      1      1
Borda1              0.869  0.426   0.18
Borda2              0.856  0.671  0.536
Borda3              0.894  0.526  0.406
Borda4              0.869  0.703  0.488
Borda5              0.856  0.668   0.52


# 4.3 Results for consensual distributive cultures

In [27]:
def SimulateTable9(n, K, simulation_runs):
    CondorChance = 0
    CondorRousseauChance = 0
    for i in range(simulation_runs):
        crc = Consensual_redistributive_culture(n,K)
        crc.setParams()
        P = crc.createProfiles()
        v = SincereVoting(crc)
        CW_exists, CW_num = v.checkCondorsetWinner()
        if(CW_exists):
            CondorChance += 1
            # We assume that truee order is 0 > 1 > 2 > ... > K -1
            # So Rousseau winner is 0
            CondorRousseauChance += int(CW_num == 0)
    # print(f'Probability for the Condorset winner  {CondorChance/simulation_runs}')
    return CondorChance/simulation_runs #, CondorRousseauChance/simulation_runs

In [28]:
# Table 9
simulation_runs = 1000

n_values = [3,5,11,99]
K_values = [3,5, 15]

print(f"\t\t\t \033[1m  Table 9  \033[0m")
df = pd.DataFrame(columns=['n = '+str(n) for n in n_values])
for K in K_values:
    line = []
    for n in n_values:    
        CondorChance  = SimulateTable9(n, K, simulation_runs)
        line.append(str(CondorChance))
    df.loc[f'K = {K}'] = line
print(df)
print('\n')


			   Table 9  
        n = 3  n = 5 n = 11 n = 99
K = 3   0.787  0.775  0.773   0.77
K = 5   0.384  0.371  0.344  0.318
K = 15  0.004  0.003  0.002  0.002




In [29]:
def SimulateTable10(n, K, simulation_runs):
    CondorNum = 0
    randomChoice = 0
    Copeland = 0 
    Borda = 0
    Plurality = 0
    
    AV_5_iter = [0]*5
    Borda_5_iter = [0]*5
    for i in range(simulation_runs):
            
        crc = Consensual_redistributive_culture(n,K)
        crc.setParams()
        P = crc.createProfiles()
        SV = SincereVoting(crc)
        RV = ResponsiveVoting(crc)
        
        CW_exists, CW_num = SV.checkCondorsetWinner()
        
        if(CW_exists):
            CondorNum += 1
        
        def getSharesForWinner(winner_index):

            shares = []
            for p in crc.SharesProfiles:
                shares.append(p[winner_index])
            return shares        
        
        randomWinner = random.randint(0,K-1)
        randomChoice += crc.simpleGiniIndex(getSharesForWinner(randomWinner))
        
        CopelandWinner = SV.CopelandWinner()
        Copeland += crc.simpleGiniIndex(getSharesForWinner(CopelandWinner))
        
        BordaWinner = SV.BordaWinner()
        Borda += crc.simpleGiniIndex(getSharesForWinner(BordaWinner))
        
        PluralityWinner = SV.PluralityWinner() 
        Plurality += crc.simpleGiniIndex(getSharesForWinner(PluralityWinner))
        
        
        #  Borda
        
        Borda_scores = RV.BordaScores()
        Borda_win = np.argmax(Borda_scores)
        Borda_5_iter[0] += crc.simpleGiniIndex(getSharesForWinner(Borda_win))
        
        for s in range(4):
            Borda_scores = RV.BordaScores(Borda_scores)
            Borda_win = np.argmax(Borda_scores)
            Borda_5_iter[s+1] += crc.simpleGiniIndex(getSharesForWinner(Borda_win))
        
        
        # AV

        AV_scores = RV.ApprovalVoting()
        AV_win = np.argmax(AV_scores)
        AV_5_iter[0] += crc.simpleGiniIndex(getSharesForWinner(AV_win))

        for s in range(4):
            AV_scores = RV.ApprovalVoting(AV_scores)
            AV_win = np.argmax(AV_scores)
            AV_5_iter[s+1] += crc.simpleGiniIndex(getSharesForWinner(AV_win))

            
    
    CondorCh = CondorNum/simulation_runs
    CopelandCh = Copeland/simulation_runs
    Plurality = Plurality/simulation_runs
    Borda = Borda/simulation_runs
    randomChoice = randomChoice/simulation_runs
    
    for s in range(5):
        Borda_5_iter[s] = Borda_5_iter[s]/simulation_runs
        AV_5_iter[s] =  AV_5_iter[s]/simulation_runs
    
    return CondorCh, randomChoice, Plurality, CopelandCh, AV_5_iter, Borda, Borda_5_iter

In [30]:
# Table 10
print(f"\t\t\t \033[1m  Table 10  \033[0m")

simulation_runs = 1000

n = 11
K_values = [3,5,15]
print(f"\t\t\t \033[1m  n = 11  \033[0m")
df = pd.DataFrame(columns=['K = 3', 'K = 5', 'K = 15'])
cols = {}
for k in K_values:
    cols[f'K = {k}']={}
    CondorCh, randomChoice, Plurality, CopelandCh, AV, Borda, Borda_5_iter  = SimulateTable10(n, k, simulation_runs)
    cols[f'K = {k}']['Pr. of Condorcet :'] = CondorCh
    cols[f'K = {k}']['Rule'] = 'Gini'
    cols[f'K = {k}']['random choice'] = randomChoice
    cols[f'K = {k}']['Plurality'] = Plurality
    cols[f'K = {k}']['Copeland:'] = CopelandCh
    for i in range(len(AV)):
        cols[f'K = {k}'][f'AV{i+1}'] = AV[i]
    cols[f'K = {k}']['Borda'] = Borda
    for i in range(len(Borda_5_iter)):
        cols[f'K = {k}'][f'Borda{i+1}'] = Borda_5_iter[i]
    for key in cols[f'K = {k}'].keys():
        df.at[key,f'K = {k}'] = cols[f'K = {k}'][key]
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 2)
print(df)


			   Table 10  
			   n = 11  
                   K = 3 K = 5 K = 15
Pr. of Condorcet :  0.76  0.35      0
Rule                Gini  Gini   Gini
random choice       0.31  0.32   0.31
Plurality           0.33  0.35    0.4
Copeland:           0.31   0.3   0.29
AV1                 0.32  0.31   0.31
AV2                 0.31  0.31   0.31
AV3                 0.31  0.31    0.3
AV4                 0.31  0.31   0.31
AV5                 0.31  0.31   0.31
Borda                0.3   0.3   0.29
Borda1               0.3  0.31   0.31
Borda2              0.31  0.31   0.31
Borda3               0.3   0.3   0.31
Borda4               0.3   0.3    0.3
Borda5              0.31   0.3    0.3


In [31]:
def SimulateTable11(n, K, simulation_runs):
    CondorChance = 0
    CondorRousseauChance = 0
    for i in range(simulation_runs):
        idc = Inegalitarian_distributive_culture(n,K)
        e_min = 1.3
        e_max = 4

        idc.setParams(e_min, e_max)
        P= idc.createProfiles()
        v = SincereVoting(idc)
        CW_exists, CW_num = v.checkCondorsetWinner()
        if(CW_exists):
            CondorChance += 1
            # We assume that truee order is 0 > 1 > 2 > ... > K -1
            # So Rousseau winner is 0
            CondorRousseauChance += int(CW_num == 0)
    # print(f'Probability for the Condorset winner  {CondorChance/simulation_runs}')
    return CondorChance/simulation_runs #, CondorRousseauChance/simulation_runs

In [32]:
# Table 11
simulation_runs = 1000

n_values = [3,5,11,99]
K_values = [3,5,15,50]

print(f"\t\t\t \033[1m  Table 11  \033[0m")
df = pd.DataFrame(columns=['n = '+str(n) for n in n_values])
for K in K_values:
    line = []
    for n in n_values:    
        CondorChance  = SimulateTable11(n, K, simulation_runs)
        line.append(str(CondorChance))
    df.loc[f'K = {K}'] = line
print(df)
print('\n')


			   Table 11  
        n = 3  n = 5 n = 11 n = 99
K = 3   0.786  0.822  0.831  0.937
K = 5   0.411  0.453  0.518  0.778
K = 15  0.015  0.011   0.03  0.212
K = 50    0.0    0.0    0.0  0.001




# 4.4 Results for inegalitarian distributive cultures

In [33]:
def SimulateTable12(n, K, simulation_runs):
    CondorNum = 0
    randomChoice = 0
    Copeland = 0 
    Borda = 0
    Plurality = 0
    
    AV_5_iter = [0]*5
    Borda_5_iter = [0]*5
    for i in range(simulation_runs):
            
        idc = Inegalitarian_distributive_culture(n,K)
        e_min = 1.3
        e_max = 4
        idc.setParams(e_min, e_max)
        P= idc.createProfiles()
        SV = SincereVoting(idc)
        RV = ResponsiveVoting(idc)
        
        CW_exists, CW_num = SV.checkCondorsetWinner()
        
        if(CW_exists):
            CondorNum += 1
        
        def getSharesForWinner(winner_index):

            shares = []
            for p in idc.SharesProfiles:
                shares.append(p[winner_index])
            return shares        
        
        randomWinner = random.randint(0,K-1)
        randomChoice += idc.simpleGiniIndex(getSharesForWinner(randomWinner))
        
        CopelandWinner = SV.CopelandWinner()
        Copeland += idc.simpleGiniIndex(getSharesForWinner(CopelandWinner))
        
        BordaWinner = SV.BordaWinner()
        Borda += idc.simpleGiniIndex(getSharesForWinner(BordaWinner))
        
        PluralityWinner = SV.PluralityWinner() 
        Plurality += idc.simpleGiniIndex(getSharesForWinner(PluralityWinner))
        
        
        #  Borda
        
        Borda_scores = RV.BordaScores()
        Borda_win = np.argmax(Borda_scores)
        Borda_5_iter[0] += idc.simpleGiniIndex(getSharesForWinner(Borda_win))
        
        for s in range(4):
            Borda_scores = RV.BordaScores(Borda_scores)
            Borda_win = np.argmax(Borda_scores)
            Borda_5_iter[s+1] += idc.simpleGiniIndex(getSharesForWinner(Borda_win))
        
        
        # AV

        AV_scores = RV.ApprovalVoting()
        AV_win = np.argmax(AV_scores)
        AV_5_iter[0] += idc.simpleGiniIndex(getSharesForWinner(AV_win))

        for s in range(4):
            AV_scores = RV.ApprovalVoting(AV_scores)
            AV_win = np.argmax(AV_scores)
            AV_5_iter[s+1] += idc.simpleGiniIndex(getSharesForWinner(AV_win))

            
    
    CondorCh = CondorNum/simulation_runs
    CopelandCh = Copeland/simulation_runs
    Plurality = Plurality/simulation_runs
    Borda = Borda/simulation_runs
    randomChoice = randomChoice/simulation_runs
    
    for s in range(5):
        Borda_5_iter[s] = Borda_5_iter[s]/simulation_runs
        AV_5_iter[s] =  AV_5_iter[s]/simulation_runs
    
    return CondorCh, randomChoice, Plurality, CopelandCh, AV_5_iter, Borda, Borda_5_iter

In [34]:
# Table 12
print(f"\t\t\t \033[1m  Table 12  \033[0m")

simulation_runs = 1000

n = 11
K_values = [3,5,15]
print(f"\t\t\t \033[1m  n = 11  \033[0m")
df = pd.DataFrame(columns=['K = 3', 'K = 5', 'K = 15'])
cols = {}
for k in K_values:
    cols[f'K = {k}']={}
    CondorCh, randomChoice, Plurality, CopelandCh, AV, Borda, Borda_5_iter  = SimulateTable12(n, k, simulation_runs)
    cols[f'K = {k}']['Pr. of Condorcet :'] = CondorCh
    cols[f'K = {k}']['Rule'] = 'Gini'
    cols[f'K = {k}']['random choice'] = randomChoice
    cols[f'K = {k}']['Plurality'] = Plurality
    cols[f'K = {k}']['Copeland:'] = CopelandCh
    for i in range(len(AV)):
        cols[f'K = {k}'][f'AV{i+1}'] = AV[i]
    cols[f'K = {k}']['Borda'] = Borda
    for i in range(len(Borda_5_iter)):
        cols[f'K = {k}'][f'Borda{i+1}'] = Borda_5_iter[i]
    for key in cols[f'K = {k}'].keys():
        df.at[key,f'K = {k}'] = cols[f'K = {k}'][key]
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 2)
print(df)


			   Table 12  
			   n = 11  
                   K = 3 K = 5 K = 15
Pr. of Condorcet :  0.82  0.54   0.04
Rule                Gini  Gini   Gini
random choice       0.42  0.43   0.42
Plurality           0.41  0.48   0.54
Copeland:           0.36  0.33   0.27
AV1                 0.42  0.43   0.36
AV2                 0.38  0.38   0.37
AV3                  0.4  0.38   0.35
AV4                 0.39   0.4   0.35
AV5                  0.4  0.39   0.35
Borda               0.36  0.33   0.26
Borda1              0.36  0.39   0.43
Borda2              0.36  0.36   0.35
Borda3              0.36  0.37   0.36
Borda4              0.36  0.35   0.35
Borda5              0.36  0.36   0.34


# 4.5 Results for spatial cultures

# 4.5.1 Results for unidimensional culture

In [35]:
def SimulateTable13(n, K, simulation_runs):
    CondorNum = 0
    Copeland = 0 
    Borda = 0
    Plurality = 0
    
    AV_5_iter = [0]*5
    Borda_5_iter = [0]*5
    for i in range(simulation_runs):
            
        udc = Spatial_culture(n,K)
        udc.setParams(1, [1])
        P = udc.createProfiles()
        SV = SincereVoting(udc)
        RV = ResponsiveVoting(udc)
        
        CW_exists, CW_num = SV.checkCondorsetWinner()
        
        if(CW_exists):
            CondorNum += 1
        
        CopelandWinner = 1 + SV.CopelandWinner()
        
        
        BordaWinner = 1 + SV.BordaWinner()
        Borda += int(BordaWinner == 1 + CW_num)
        
        PluralityWinner = 1 + SV.PluralityWinner() 
        Plurality += int(PluralityWinner == 1 + CW_num)
        
        #  Borda
        
        Borda_scores = RV.BordaScores()
        Borda_win = 1 + np.argmax(Borda_scores)
        Borda_5_iter[0] += int(Borda_win == 1 + CW_num)
        
        for s in range(4):
            Borda_scores = RV.BordaScores(Borda_scores)
            Borda_win = 1 + np.argmax(Borda_scores)
            Borda_5_iter[s+1] += int(Borda_win == 1 + CW_num)
                
    CondorCh = CondorNum/simulation_runs
    Plurality = Plurality/simulation_runs
    Borda = Borda/simulation_runs
    
    for s in range(5):
        Borda_5_iter[s] = Borda_5_iter[s]/simulation_runs

    
    return CondorCh, Plurality,Borda, Borda_5_iter

In [36]:
# Table 13
print(f"\t\t\t \033[1m  Table 13  \033[0m")

simulation_runs = 10000

n = 11
K_values = [3,5,15]
print(f"\t\t\t \033[1m  n = 11  \033[0m")
df = pd.DataFrame(columns=['K = 3', 'K = 5', 'K = 15'])
cols = {}
for k in K_values:
    cols[f'K = {k}']={}
    CondorCh, Plurality, Borda, Borda_5_iter  = SimulateTable13(n, k, simulation_runs)
    cols[f'K = {k}']['Pr. of Condorcet :'] = CondorCh
    cols[f'K = {k}']['Rule'] = 'Condorcet'
    cols[f'K = {k}']['Plurality'] = Plurality
    cols[f'K = {k}']['Copeland:'] = 1
    for i in range(len(AV)):
        cols[f'K = {k}'][f'AV{i+1}'] = 1
    cols[f'K = {k}']['Borda'] = Borda
    for i in range(len(Borda_5_iter)):
        cols[f'K = {k}'][f'Borda{i+1}'] = Borda_5_iter[i]
    for key in cols[f'K = {k}'].keys():
        df.at[key,f'K = {k}'] = cols[f'K = {k}'][key]
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)
print(df)


			   Table 13  
			   n = 11  
                        K = 3      K = 5     K = 15
Pr. of Condorcet :          1          1          1
Rule                Condorcet  Condorcet  Condorcet
Plurality              0.7576     0.5333     0.2983
Copeland:                   1          1          1
AV1                         1          1          1
AV2                         1          1          1
AV3                         1          1          1
AV4                         1          1          1
AV5                         1          1          1
Borda                  0.8633     0.7822     0.6102
Borda1                      1     0.5166     0.0963
Borda2                      1     0.8803       0.13
Borda3                      1     0.7641     0.1894
Borda4                      1     0.8757     0.2571
Borda5                      1     0.8336     0.3051


# 4.5.2 Results for multi-dimensional culture

In [37]:
def SimulateTable14(n, K, simulation_runs):
    CondorNum = 0
    Copeland = 0 
    Borda = 0
    Plurality = 0
    
    AV_5_iter = [0]*5
    Borda_5_iter = [0]*5
    for i in range(simulation_runs):
            
        mdc = Spatial_culture(n,K)
        mdc.setParams(1, [1, 0.5])
        P = mdc.createProfiles()
        SV = SincereVoting(mdc)
        RV = ResponsiveVoting(mdc)
        
        CW_exists, CW_num = SV.checkCondorsetWinner()
        
        if(CW_exists):
            CondorNum += 1
        
        CopelandWinner = 1 + SV.CopelandWinner()
        
        PluralityWinner = 1 + SV.PluralityWinner() 
        Plurality += int(PluralityWinner == CopelandWinner)
        
        BordaWinner = 1 + SV.BordaWinner()
        Borda += int(BordaWinner == CopelandWinner)
        
        
        # AV

        AV_scores = RV.ApprovalVoting()
        AV_win = 1 + np.argmax(AV_scores)
        AV_5_iter[0] += int(AV_win == CopelandWinner)

        for s in range(4):
            AV_scores = RV.ApprovalVoting(AV_scores)
            AV_win = 1 + np.argmax(AV_scores)
            AV_5_iter[s+1] += int(AV_win == CopelandWinner)
            
        #  Borda
        
        
            
        Borda_scores = RV.BordaScores()
        Borda_win = 1 + np.argmax(Borda_scores)
        Borda_5_iter[0] += int(Borda_win == CopelandWinner)

        for s in range(4):
            Borda_scores = RV.BordaScores(Borda_scores)
            Borda_win = 1 + np.argmax(Borda_scores)
            Borda_5_iter[s+1] += int(Borda_win == CopelandWinner)
                
    CondorCh = CondorNum/simulation_runs
    Plurality = Plurality/simulation_runs
    Borda = Borda/simulation_runs
    
    for s in range(5):
        AV_5_iter[s] = AV_5_iter[s]/simulation_runs
        Borda_5_iter[s] = Borda_5_iter[s]/simulation_runs

    
    return CondorCh, Plurality,AV_5_iter, Borda, Borda_5_iter


In [38]:
# Table 14
print(f"\t\t\t \033[1m  Table 14  \033[0m")

simulation_runs = 1000

n = 11
K_values = [3,5,15]
print(f"\t\t\t \033[1m  n = 11  \033[0m")
df = pd.DataFrame(columns=['K = 3', 'K = 5', 'K = 15'])
cols = {}
for k in K_values:
    cols[f'K = {k}']={}
    CondorCh, Plurality, AV_5_iter, Borda, Borda_5_iter  = SimulateTable14(n, k, simulation_runs)
    cols[f'K = {k}']['Pr. of Condorcet :'] = CondorCh
    cols[f'K = {k}']['Rule'] = 'Copeland'
    cols[f'K = {k}']['Plurality'] = Plurality
    cols[f'K = {k}']['Copeland:'] = 1
    for i in range(len(AV)):
        cols[f'K = {k}'][f'AV{i+1}'] = AV_5_iter[i]
    cols[f'K = {k}']['Borda'] = Borda
    for i in range(len(Borda_5_iter)):
        cols[f'K = {k}'][f'Borda{i+1}'] = Borda_5_iter[i]
    for key in cols[f'K = {k}'].keys():
        df.at[key,f'K = {k}'] = cols[f'K = {k}'][key]
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)
print(df)


			   Table 14  
			   n = 11  
                       K = 3     K = 5    K = 15
Pr. of Condorcet :     0.988     0.958     0.807
Rule                Copeland  Copeland  Copeland
Plurality              0.777     0.573     0.266
Copeland:                  1         1         1
AV1                    0.756     0.392     0.075
AV2                    0.929     0.746     0.359
AV3                    0.792     0.559     0.401
AV4                    0.915     0.701      0.39
AV5                    0.803     0.536     0.298
Borda                  0.901     0.805     0.633
Borda1                 0.991     0.471     0.093
Borda2                 0.992     0.896     0.249
Borda3                 0.993     0.712     0.353
Borda4                 0.991     0.887     0.388
Borda5                 0.992     0.815      0.43
